**Matthew Woo
20236203**


In [ ]:
import numpy as np
from enum import IntEnum
from copy import deepcopy
import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')
plt.style.use('seaborn-whitegrid')
import matplotlib.colors as mcolors

<ipython-input-2-b53f5efa330b>:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-notebook')
<ipython-input-2-b53f5efa330b>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [ ]:
class Action(IntEnum):
    up = 0
    right = 1
    down = 2
    left = 3

action_to_str = {
    Action.up : "up",
    Action.right : "right",
    Action.down : "down",
    Action.left : "left",
}

action_to_offset = {
    Action.up : (-1, 0),
    Action.right : (0, 1),
    Action.down : (1, 0),
    Action.left : (0, -1),
}

In [ ]:
class GridWorld:

    def __init__(self, height, width, goal, goal_value=1.0, danger=[], danger_value=-1.0, blocked=[], noise=0.0):
        """
        Initialize the GridWorld environment.
        Creates a gridworld like MDP
         - height (int): Number of rows
         - width (int): Number of columns
         - goal (int): Index number of goal cell
         - goal_value (float): Reward given for goal cell
         - danger (list of int): Indices of cells marked as danger
         - danger_value (float): Reward given for danger cell
         - blocked (list of int): Indices of cells marked as blocked (can't enter)
         - noise (float): probability of resulting state not being what was expected
        """

        self._width = width
        self._height = height
        self._grid_values = [[0,0,0,0] for _ in range(height*width)] # Initialize state values.
        self._grid_policies = [np.random.randint(0,4) for _ in range(height * width)]
        self._returns = [[[],[],[],[]] for _ in range(height*width)]
        self._goal_value = goal_value
        self._danger_value = danger_value
        self._goal_cell = goal
        self._danger_cells = danger
        self._blocked_cells = blocked
        self._noise = noise # Noise level in the environment.
        assert noise >= 0 and noise < 1 # Ensure valid noise value.
        self.create_next_values() # Initialize the next state values.


    def reset(self):
        """
        Reset the state values to their initial state.
        """
        self._grid_values = [[0,0,0,0] for _ in range(self.height*self.width)]
        self.create_next_values()


    def _inbounds(self, state):
        """
        Check if a state index is within the grid boundaries.
        """
        return state >= 0 and state < self._width * self._height

    def _inbounds_rc(self, state_r, state_c):
        """
        Check if row and column indices are within the grid boundaries.
        """
        return state_r >= 0 and state_r < self._height and state_c >= 0 and state_c < self._width

    def _state_to_rc(self, state):
        """
        Convert a state index to row and column indices.
        """
        return state // self._width, state % self._width

    def _state_from_action(self, state, action):
        """
        Gets the state as a result of applying the given action
        """
        #TO DO:
        #[1,2,3,4,5,6,7,8,9,10,11,12]
        a = self.get_actions(state)

        if action in a:
            if action==0:
                return state-self._width
            elif action==1:
                return state+1
            elif action==2:
                return state+self._width
            else:
                return state-1
        return state

    def is_terminal(self, state):
        """
        Returns true if a state is terminal (goal, or danger)
        """
        #To Do:
        return (state == self._goal_cell) or (state in self._danger_cells)

    def get_states(self):
        """
        Gets all non-terminal states in the environment
        """
        #TO DO:
        non_term = []
        for x in range(self._width*self._height):
            if not self.is_terminal(x) and x not in self._blocked_cells:
                non_term.append(x)
        return non_term

    def get_actions(self, state):
        """
        Returns a list of valid actions given the current state
        """
        #TO DO:
        #Returns only actions that go to non block states

        #Obtaining row and col of all block states
        block = []
        for i in self._blocked_cells:
            r,c = self._state_to_rc(i)
            block.append([r,c])

        #Loop through all actions, check if the taking action is within bounds
        valid_act = []
        r,c = self._state_to_rc(state)
        for x in range(0,4):
            a = action_to_offset[x]
            if self._inbounds_rc(r+a[0],c+a[1]) and [r+a[0],c+a[1]] not in block:
                valid_act.append(x)
        return valid_act


    def get_reward(self, state):
        """
        Get the reward for being in the current state
        """
        assert self._inbounds(state)
        # Reward is non-zero for danger or goal
        #TO DO:
        if state == self._goal_cell:
            return 1
        elif state in self._danger_cells:
            return -1
        return -0.1

    def __str__(self):
        """
        Pretty print the state values
        """
        out_str = ""
        for r in range(self._height):
            for c in range(self._width):
                cell = r * self._width + c
                if cell in self._blocked_cells:
                    out_str += "{:>6}".format("----")
                elif cell == self._goal_cell:
                    out_str += "{:>6}".format("GOAL")
                elif cell in self._danger_cells:
                    out_str += "{:>6.2f}".format(self._danger_value)
                else:
                    out_str += "{:>6.2}".format(action_to_str[self._grid_policies[cell]])
                out_str += " "
            out_str += "\n"
        return out_str

In [ ]:
def mc_on_policy_explore(gw, discount):
    #Initialize action state estimate and policy for all states
    #Already done with initialization of gridworld

    #Randomly choosing an action different from the initial policy is an exploring start
    episode_count = 0
    while episode_count<4000:
        #Can choose the starting state or make it random if you uncomment line below
        s = np.random.choice(gw.get_states(), 1)[0]
        a = np.random.randint(0,4)
        iter = 0
        episode = []
        #Episode generation
        while iter < 30 and not gw.is_terminal(s):
            nextState = gw._state_from_action(s,a)
            rew = gw.get_reward(nextState)
            #Tuple storing the current state, action and reward for taking action
            episode.append((s,a,rew))
            s = nextState
            a = gw._grid_policies[s]
            iter += 1
        #Episode traversal
        g = 0
        for t in range(len(episode)-1,-1,-1):
            g = discount*g + episode[t][2]
            ts = episode[t][0]
            ta = episode[t][1]
            prev = False
            #First Visit
            #If the state and action at the current t iteration show up in an earlier t then skip the current iteration
            for check in range(0,t):
                if ts == episode[check][0] and ta == episode[check][1]:
                    prev = True
                    break
            if not prev:
                gw._returns[ts][ta].append(g)
                gw._grid_values[ts][ta] = np.mean(gw._returns[ts][ta])
                gw._grid_policies[ts] = np.random.choice(np.where(gw._grid_values[ts] == np.array(gw._grid_values[ts]).max())[0])
        episode_count += 1

In [ ]:
test_gw = GridWorld(height=3,width=4,goal=3,danger=[7],blocked=[5],noise=0.0)
mc_on_policy_explore(test_gw,0.95)
print(test_gw)
for x in test_gw._grid_values:
  print(x)

    ri     ri     ri   GOAL 
    up   ----     up  -1.00 
    ri     ri     up     le 

[0.5356882359818034, 0.705138084902602, 0.426349373551596, 0.5507082937574255]
[0.6879224014957074, 0.8499999999999996, 0.7074999999999998, 0.5515206973835179]
[0.8499999999999995, 1.0, 0.7074999999999997, 0.7074999999999997]
[0, 0, 0, 0]
[0.5639927515289785, 0.4206262480687948, 0.28954339595186346, 0.364328497049916]
[0, 0, 0, 0]
[0.8499999999999999, -1.0, 0.5721249999999999, 0.686788886617144]
[0, 0, 0, 0]
[0.37178329165752455, 0.4361998726478665, 0.30349314000364036, 0.290255729730339]
[0.4266403799320775, 0.5700533881131529, 0.40806840583956644, 0.30336712278351097]
[0.7074999999999998, 0.4435187500000001, 0.5721249999999999, 0.39961203665629896]
[-1.0, 0.4435187500000001, 0.4435187500000001, 0.5721249999999999]


In [ ]:
def mc_on_policy_no_exp(gw, discount, epsilon):
    episode_count = 0
    gw._grid_policies = [[(1-epsilon)+(epsilon/4),epsilon/4,epsilon/4,epsilon/4] for _ in range(gw._height*gw._width)]
    while episode_count < 4000:
        #Can choose the starting state or make it random if you uncomment line below
        #s = np.random.choice(gw.get_states(), 1)[0]
        s = 8
        a = np.random.choice([0,1,2,3],1,[gw._grid_policies[s]])[0]
        iter=0
        episode=[]
        #Episode Generation
        while iter < 30 and not gw.is_terminal(s):
            nextState = gw._state_from_action(s,a)
            rew = gw.get_reward(nextState)
            #Tuple storing the current state, action and reward for taking action
            episode.append((s,a,rew))
            s = nextState
            a = np.random.choice([0,1,2,3],1,[gw._grid_policies[s]])[0]
            iter += 1
        g = 0
        for t in range(len(episode)-1,-1,-1):
            g = (g*discount) + episode[t][2]
            ts = episode[t][0]
            ta = episode[t][1]
            prev = False
            #First Visit
            #If the state and action at the current t iteration show up in an earlier t then skip the current iteration
            for check in range(0,t):
                if ts == episode[check][0] and ta == episode[check][1]:
                    prev = True
                    break
            if not prev:
                gw._returns[ts][ta].append(g)
                gw._grid_values[ts][ta] = sum(gw._returns[ts][ta])/len(gw._returns[ts][ta])
                #Choosing the action/index with the greatest state action value
                A = np.random.choice(np.where(gw._grid_values[ts] == np.array(gw._grid_values[ts]).max())[0])
                #action equal to A will be the greedy action while the rest are non greedy
                for x in range(4):
                    if x == A:
                        gw._grid_policies[ts][x] = 1 - epsilon + epsilon/4
                    else:
                        gw._grid_policies[ts][x] = epsilon/4

        episode_count += 1
    #Finalizing policy
    for x in range(gw._height*gw._width):
          gw._grid_policies[x] = np.random.choice(np.where(gw._grid_policies[x] == np.array(gw._grid_policies[x]).max())[0])

In [ ]:
tiny_gw2 = GridWorld(height=3,width=4,goal=3,danger=[7],blocked=[5],noise=0.0)
mc_on_policy_no_exp(tiny_gw2,0.95,0.25)
print(tiny_gw2)
for x in tiny_gw2._grid_values:
  print(x)

    ri     ri     ri   GOAL 
    up   ----     up  -1.00 
    ri     ri     up     le 

[-0.9484234635345208, -0.6893232965213318, -1.0773279005849377, -0.9346779375753211]
[-0.6123315052003212, -0.23904415044948063, -0.600420441262076, -0.8604526945993399]
[-0.17429856773458044, 1.0, -0.7488913228116552, -0.6358650397996026]
[0, 0, 0, 0]
[-1.0296879753744936, -1.1365010759443033, -1.2066473344775723, -1.1501596668587186]
[0, 0, 0, 0]
[-0.2177473525778785, -1.0, -0.9552616652760914, -0.7726254593579693]
[0, 0, 0, 0]
[-1.2642113943142512, -1.2639810225401198, -1.2904598535678977, -1.292976241390617]
[-1.1829496151576884, -1.0793197688634775, -1.1783284120501356, -1.2206759232276572]
[-0.8094941656520858, -1.048403075329322, -1.0183028110227192, -1.09714720558039]
[-1.0, -1.0342010347036497, -1.0328177301398918, -0.9832135880787689]


In [ ]:
def off_policy_mc_prediction(gw, discount, epsilon, policy=None):
    C = [[0,0,0,0] for _ in range(gw._height*gw._width)]
    gw._grid_policies = [[1,0,0,0] for _ in range(gw._height*gw._width)]
    if policy != None:
        gw._grid_policies = policy.copy()
    episode_count = 0

    #Episode generation
    while episode_count < 4000:
        b = [[0.25,0.25,0.25,0.25] for _ in range(gw._height*gw._width)]
        episode = []
        iter = 0
        #Can choose the starting state or make it random if you uncomment line below
        #s = np.random.choice(gw.get_states(), 1)[0]
        s=8
        a = np.random.choice([0,1,2,3],1,b[s])[0]
        while iter<30 or not gw.is_terminal(s):
            newstate = gw._state_from_action(s,a)
            rew = gw.get_reward(newstate)
            #Tuple storing the current state, action and reward for taking action
            episode.append((s,a,rew))
            s=newstate
            a=np.random.choice([0,1,2,3],1,b[s])[0]
            iter+=1
        #Episode traversal
        g=0
        w=1
        for t in range(len(episode)-1,-1,-1):
            if w == 0:
                break
            ts = episode[t][0]
            ta = episode[t][1]
            g = (discount*g) + episode[t][2]
            C[ts][ta] = C[ts][ta] + w
            gw._grid_values[ts][ta] = gw._grid_values[ts][ta] + (w/C[ts][ta])*(g - gw._grid_values[ts][ta])
            w = w*(gw._grid_policies[ts][ta]/b[ts][ta])
        episode_count+=1

    #Finalizing policy
    for x in range(len(gw._grid_policies)):
        gw._grid_policies[x] = np.random.choice(np.where(gw._grid_values[x] == np.array(gw._grid_values[x]).max())[0])

In [ ]:
tiny_gw3 = GridWorld(height=3,width=4,goal=3,danger=[7],blocked=[5],noise=0.0)
policy = [[0,1,0,0],[0,1,0,0],[0,1,0,0],[0,0,0,0],[1,0,0,0],[0,0,0,0],[1,0,0,0],[0,0,0,0],[1,0,0,0],[0,1,0,0],[1,0,0,0],[0,0,0,1]]
off_policy_mc_prediction(tiny_gw3,0.95,0.25,policy)
print(tiny_gw3)
for x in tiny_gw3._grid_values:
  print(x)

    ri     ri     ri   GOAL 
    up   ----     up  -1.00 
    up     ri     up     le 

[0.572125, 0.7075, 0.44351874999999996, 0.572125]
[0.7075, 0.85, 0.7075, 0.572125]
[0.85, 1.0, 0.7075, 0.7075]
[1.0, 1.0, -1.0, 0.85]
[0.572125, 0.44351874999999996, 0.32134281249999996, 0.44351874999999996]
[0, 0, 0, 0]
[0.85, -1.0, 0.572125, 0.7075]
[1.0, -1.0, 0.44351874999999996, 0.7075]
[0.44351874999999996, 0.44351874999999996, 0, 0.32134281249999996]
[0.44351874999999996, 0.572125, 0.44351874999999996, 0.32134281249999996]
[0.7075, 0.44351874999999996, 0.572125, 0.44351874999999996]
[-1.0, 0.44351874999999996, 0.44351874999999996, 0.572125]


In [ ]:
def off_policy_mc_estimate(gw, discount, epsilon):
    C = [[0,0,0,0] for _ in range(gw._height*gw._width)]

    episode_count = 0
    while episode_count < 4000:
        #Creating random epsilon soft policy b
        b = []
        for state in range(gw._height*gw._width):
            temp = [0,0,0,0]
            #Just appending an all 0 list to keep state index/order correct
            if gw.is_terminal(state):
                b.append(temp)
                continue
            #maxact = np.random.choice(np.where(gw._grid_values[x] == np.array(gw._grid_values[x]).max())[0])
            maxact = np.random.randint(0,4)
            for act in range(4):
                if maxact == act:
                    temp[act] = (1-epsilon) + epsilon/4
                else:
                    temp[act] = epsilon/4
            b.append(temp)
        episode = []
        iter = 0
        #Can choose the starting state or make it random if you uncomment line below
        #s = np.random.choice(gw.get_states(), 1)[0]
        s=8
        a = np.random.choice(np.where(b[s] == np.array(b[s]).max())[0])

        while iter<30 and not gw.is_terminal(s):
            newstate = gw._state_from_action(s,a)
            rew = gw.get_reward(newstate)
            #Tuple storing the current state, action and reward for taking action
            episode.append((s,a,rew))
            s = newstate
            #Selecting action based on b policy
            a = np.random.choice([0,1,2,3],1,b[s])[0]
            iter+=1

        g = 0
        w = 1
        for t in range(len(episode)-1,-1,-1):
            ts = episode[t][0]
            ta = episode[t][1]
            g = (discount*g) + episode[t][2]
            C[ts][ta] = C[ts][ta] + w
            gw._grid_values[ts][ta] = gw._grid_values[ts][ta] + (w/C[ts][ta])*(g-gw._grid_values[ts][ta])
            gw._grid_policies[ts] = np.random.choice(np.where(gw._grid_values[ts]==np.array(gw._grid_values[ts]).max())[0])
            if ta != gw._grid_policies[ts]:
                break
            w = w*(1/b[ts][ta])
        episode_count+=1


In [ ]:
tiny_gw4 = GridWorld(height=3,width=4,goal=3,danger=[7],blocked=[5],noise=0.0)
off_policy_mc_estimate(tiny_gw4,0.95,0.25)
print(tiny_gw4)
for x in tiny_gw4._grid_values:
  print(x)

    ri     ri     ri   GOAL 
    up   ----     up  -1.00 
    ri     ri     up     le 

[0.566966771178729, 0.6977268276372671, 0.4398867384420996, 0.5391836473725828]
[0.6971176657770882, 0.8405606077149852, 0.6857862269460351, 0.5693336176944059]
[0.8410572049239677, 1.0, 0.6935869812225934, 0.6985047198080485]
[0, 0, 0, 0]
[0.5662339947000565, 0.44151419768367606, 0.3203595816891152, 0.4420906531754437]
[0, 0, 0, 0]
[0.8397374106697696, -1.0, 0.5367816522145943, 0.6931820680238764]
[0, 0, 0, 0]
[0.42440771148415046, 0.4323545621949361, 0.32059525309300374, 0.2751741859878291]
[0.4338761788058121, 0.5575706765689029, 0.4091198859548373, 0.314322534605197]
[0.693102711563939, 0.4430415489908973, 0.5623045361127288, 0.439869909431457]
[-1.0, 0.40553908195395316, 0.3989086348369919, 0.5607596072860089]


In [ ]:
#print(tiny_gw3._grid_values)
#print(tiny_gw4._grid_values)

#Calculating the difference between different algorithms final _grid_values

for x in tiny_gw3.get_states():
    print(np.subtract(tiny_gw3._grid_values[x],tiny_gw4._grid_values[x]))

for x in tiny_gw3.get_states():
    print(np.subtract(test_gw._grid_values[x],tiny_gw3._grid_values[x]))

[0.00475566 0.00764255 0.07973654 0.00680764]
[0.00902498 0.00997288 0.01419473 0.01040637]
[0.00878527 0.         0.00780841 0.01679128]
[0.00913365 0.00211929 0.31245853 0.01399286]
[0.01012888 0.         0.02091808 0.02574852]
[0.00746041 0.0015551  0.00018705 0.15685407]
[0.0194314  0.00740088 0.00559358 0.00024146]
[0.00880727 0.00216908 0.00603008 0.01258298]
[0.         0.00440608 0.01634396 0.0156649 ]
[-0.09777597 -0.01067187 -0.16013694 -0.08349624]
[-0.10247476 -0.00206557 -0.06533799 -0.20799677]
[-0.02122238  0.         -0.03594006 -0.11148799]
[-0.02172791 -0.11605391 -0.03044121 -0.11757464]
[-0.00204799  0.         -0.0338382  -0.08269097]
[-0.12934825 -0.01011963 -0.04835183 -0.04364201]
[-0.03194457 -0.01237244 -0.02032294 -0.02655392]
[-0.01124385 -0.05280323 -0.05594761 -0.03653308]
[ 0.         -0.02928711 -0.0676976  -0.00845955]
